In [ ]:
# import nltk
# nltk.download()

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import json

In [2]:
#read in csv
types = pd.read_csv("../mbti_1.csv")
types.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


# EDA

In [3]:
types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    8675 non-null   object
 1   posts   8675 non-null   object
dtypes: object(2)
memory usage: 135.7+ KB


In [4]:
# check for null
types.isnull().sum()

type     0
posts    0
dtype: int64

In [5]:
# unique check 
types.nunique()

type       16
posts    8675
dtype: int64

In [6]:
types.type.value_counts()

INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: type, dtype: int64

In [7]:
# add plots

# Preprocessing

In [8]:
# removing piping
types['posts']= types['posts'].str.replace('|',' ')
types.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw ...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired. That's another silly misconce...


In [9]:
# removing '
types['posts']= types['posts'].str.replace("'",'')
types.head()

,type,posts
0,INFJ,http://www.youtube.com/watch?v=qsXHcwe3krw h...
1,ENTP,Im finding the lack of me in these posts very ...
2,INTP,Good one _____ https://www.youtube.com/watc...
3,INTJ,"Dear INTP, I enjoyed our conversation the ot..."
4,ENTJ,Youre fired. Thats another silly misconcepti...


In [10]:
# removing url's from posts
types['posts'] = types['posts'].str.replace('http\S+|www.\S+', ' ', case=False)
types.head()

,type,posts
0,INFJ,enfp and intj moments sportscenter...
1,ENTP,Im finding the lack of me in these posts very ...
2,INTP,"Good one _____ Of course, to which I sa..."
3,INTJ,"Dear INTP, I enjoyed our conversation the ot..."
4,ENTJ,Youre fired. Thats another silly misconcepti...


In [11]:
# change case to lower
types['posts'] = types['posts'].str.lower()
types.head()

,type,posts
0,INFJ,enfp and intj moments sportscenter...
1,ENTP,im finding the lack of me in these posts very ...
2,INTP,"good one _____ of course, to which i sa..."
3,INTJ,"dear intp, i enjoyed our conversation the ot..."
4,ENTJ,youre fired. thats another silly misconcepti...


In [12]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
#remove punctuation from posts

def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

types['body_text_clean'] = types['posts'].apply(lambda x: remove_punct(x))


In [29]:
# apply word_tokenize to all records
from nltk.tokenize import word_tokenize

def find_tokens(row):
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

types['tokenized'] = types['body_text_clean'].apply(word_tokenize)

types.head()

,type,posts,body_text_clean,tokenized
0,INFJ,enfp and intj moments sportscenter...,enfp and intj moments sportscenter...,"[enfp, and, intj, moments, sportscenter, not, ..."
1,ENTP,im finding the lack of me in these posts very ...,im finding the lack of me in these posts very ...,"[im, finding, the, lack, of, me, in, these, po..."
2,INTP,"good one _____ of course, to which i sa...",good one of course to which i say i kn...,"[good, one, of, course, to, which, i, say, i, ..."
3,INTJ,"dear intp, i enjoyed our conversation the ot...",dear intp i enjoyed our conversation the oth...,"[dear, intp, i, enjoyed, our, conversation, th..."
4,ENTJ,youre fired. thats another silly misconcepti...,youre fired thats another silly misconceptio...,"[youre, fired, thats, another, silly, misconce..."


In [35]:
# types['body_text_clean'].iloc[0]

In [ ]:
# remove instances of mbti type